This notebook computes the Summer Days index: starting from the daily maximum temperature (2096-2100),
the Summer Days index is the annual count of days number of days where TX (daily maximum temperature) > 25°C

Connect to the remote ECAS instance

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

Import source data (minimum temperature °K)

In [ ]:
maxtemp = cube.Cube(src_path='/data-training/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
    measure='tasmax',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    description='Max Temps'
    )

Identify the summer days

In [ ]:
summerdays = maxtemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-298.15','>0','1','0')"
)

Count the number of summer days

In [ ]:
count = summerdays.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)

Plot the result

In [ ]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

In [ ]:
data = firstyear.export_array(show_time='yes')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Summer Days (year 2096)')
plt.show()

To clear your workspace before running other notebooks

In [ ]:
cube.Cube.client.submit("oph_delete cube=[container=tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc]")
cube.Cube.deletecontainer(delete_type="physical", hidden="no", container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc')